In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random 
import re
import preprocessor as p


In [2]:
#chunksize = 10 ** 8

#btcdata = pd.read_csv('../data/interim/copy/bitcoin_engage_tweets_search_result_temp.csv')
#btcdata = pd.read_csv('../data/raw/btc-tweets-2016-2019/tweets.csv', sep='delimiter', header=None, low_memory=False)
#btcdata = pd.read_csv('../data/interim/bitcoin_all_tweets_search_result.csv')
btcTrainData = pd.read_csv('../data/interim/bitcoin_all_tweets_search_result_20210101-20210930.csv')



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,4,5,6,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:

btcTrainData = btcTrainData[['id', 'date', 'text', 'hashtags', 'replyCount', 'retweetCount',
'likeCount', 'userName', 'userFollowerCount', 'userFavCount', 'userFriendCount']]
print(btcTrainData.shape)
btcTrainData.info()


(20357616, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20357616 entries, 0 to 20357615
Data columns (total 11 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   id                 object
 1   date               object
 2   text               object
 3   hashtags           object
 4   replyCount         object
 5   retweetCount       object
 6   likeCount          object
 7   userName           object
 8   userFollowerCount  object
 9   userFavCount       object
 10  userFriendCount    object
dtypes: object(11)
memory usage: 1.7+ GB


In [4]:
#Remove duplicates based on the id
print('before dropping duplicate by id:' , btcTrainData.shape)
btcTrainData = btcTrainData.drop_duplicates(subset=['id'], keep='first')
print('after dropping duplicate by id:' , btcTrainData.shape)


before dropping duplicate by id: (20357616, 11)
after dropping duplicate by id: (20357288, 11)


In [5]:
#convert type of numeric fields
btcTrainData['replyCount'] = pd.to_numeric(btcTrainData['replyCount'], errors='coerce')
btcTrainData['retweetCount'] = pd.to_numeric(btcTrainData['retweetCount'], errors='coerce')
btcTrainData['likeCount'] = pd.to_numeric(btcTrainData['likeCount'], errors='coerce')
btcTrainData['userFollowerCount'] = pd.to_numeric(btcTrainData['userFollowerCount'], errors='coerce')
btcTrainData['userFavCount'] = pd.to_numeric(btcTrainData['userFavCount'], errors='coerce')
btcTrainData['userFriendCount'] = pd.to_numeric(btcTrainData['userFriendCount'], errors='coerce')

#convert type of text fields
btcTrainData['text'] = btcTrainData['text'].astype("string")

#btcdata.shape
btcTrainData = btcTrainData.dropna(subset=['replyCount', 'retweetCount', 'likeCount', 'userFollowerCount', 'userFavCount', 'userFriendCount'])
btcTrainData.shape
#btcdata = btcdata.astype({
    #"userFollowerCount": "int",
    #"userFavCount": "int",
    #"userFriendCount": "int",
    #"replyCount": "int",
    #"retweetCount": "int",
    #"likeCount": "int"
    #}) 


#btcdata.describe().apply(lambda s: s.apply('{0:.5f}'.format))



(20357287, 11)

In [6]:
#converting to datetime from string
btcTrainData["dateTime"] = pd.to_datetime(btcTrainData["date"], format="%Y-%m-%d %H:%M:%S", errors='coerce')

#to get rows from a period of time using dateTime
#august = btcdata[(btcdata['dateTime'] > '2021-08-01') & (btcdata['dateTime'] < '2021-09-01')]


In [7]:
btcTrainData.text[30000:40000]
#data = btcdata.astype({"text": "str"}, copy=True)
#data['text'] = data['text'].str.replace('@[\w]+', '')
#data.info()


30000    It appears we have a #vet breakout on the 4 HR...
30001    @CryptoWhale @XRPisOurFuture And the number of...
30002    @DocumentingBTC It was inevitable ¯\_(ツ)_/¯

#...
30003             @IAMLLUCIANA @Nois3NFT @honey_btc Oh wow
30004    @Bitcoin_Fool @JoWeb5 @EricBalchunas @DigiEcon...
                               ...                        
39995    Most of u aint even ready for #bitcoin @ 100k ...
39996        #Bitcoin fixes this 😛 https://t.co/4CBpJmY1NS
39997    $BTC go to 45k-47k

New idea by @cryptodailyuk...
39998    @0xjustblaze @insiliconot Would need btc to br...
39999    @permabearXBT What is the point analyzing alts...
Name: text, Length: 10000, dtype: string

In [8]:
btcTrainData['text'] = btcTrainData['text'].apply(lambda x: x.lower()) #transform text to lowercase


data = btcTrainData
data.text[39999]

'@permabearxbt what is the point analyzing alts/usd charts? if btc decides to move against, alts will follow. btc/usd + alts/btc makes more sence imo. do you agree?'

In [9]:
#dropping the tweets from users with no followers, friends and favourites
print('before dropping invalid users:' , data.shape)
data = data[~((data["userFollowerCount"] == 0) & (data["userFavCount"] == 0) & (data["userFriendCount"] == 0))]
#data = data.drop(data[(data["user_followers"] == 0) | (data["user_friends"] == 0) | (data["user_favourites"] == 0)].index)
print('after dropping invalid users:', data.shape)


before dropping invalid users: (20357287, 12)
after dropping invalid users: (20350599, 12)


In [10]:
#dropping the tweets with suspicious hashtags, words, bigrams, and trigrams

#any(substring in string for substring in substring_list)

#hashtags
print('before dropping by hashtags:' , data.shape)
data = data[~data.text.str.contains('#freecrypto|#freemoney|#freebitcoin|#freebtc')]
print('after dropping by hashtags:' , data.shape)

#words
print('before dropping by single keyword:' , data.shape)
data = data[~data.text.str.contains('entertaining|subscribe')]
print('after dropping by single keyword:' , data.shape)

#bigrams
print('before dropping by bigrams:' , data.shape)
data = data[~(data.text.str.contains('free bitcoin|current price|bitcoin price|earn bitcoin'))]
print('after dropping by bigrams:' , data.shape)
#data = data[~(data.text.str.contains('current') & (data.text.str.contains('price')))]
#data = data[~(data.text.str.contains('bitcoin') & (data.text.str.contains('price')))]
#data = data[~(data.text.str.contains('earn') & (data.text.str.contains('bitcoin')))]

#trigrams
print('before dropping by trigrams:' , data.shape)
data = data[~(data.text.str.contains('start trading bitcoin'))]
print('after dropping by trigrams:' , data.shape)


before dropping by hashtags: (20350599, 12)
after dropping by hashtags: (20330679, 12)
before dropping by single keyword: (20330679, 12)
after dropping by single keyword: (20298562, 12)
before dropping by bigrams: (20298562, 12)
after dropping by bigrams: (19936961, 12)
before dropping by trigrams: (19936961, 12)
after dropping by trigrams: (19936535, 12)


In [11]:
#Remove urls, mentions, hashtags, empty lines, spaces, reserved words (RT, FAV)
#keep emojis, smileys, numbers
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.RESERVED)
data['text'] = data['text'].apply(lambda x: p.clean(x))



In [12]:
#Check the header of formated text
data.text[100:200]


101    that is going to be a lovely office. anyway, l...
102     cryptocurrencies are eating straws in your heart
103                                 join me in my space!
104    american hodlers should be taking advantage of...
105    october is projected to be the start of the cr...
                             ...                        
198    is very high quality and reliable platform. th...
199    just reporting the stats, it is the part of sc...
200    cryptocurrency ransomware payments rose by 300...
201    probably the most intelligent take on bitcoin ...
202    forex trading: what is forex? fx trading expla...
Name: text, Length: 100, dtype: object

In [13]:
#remove special characters
#data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x)) 

In [14]:
#remove extra space between words
#data['text'] = data['text'].apply(lambda x: re.sub(' +', ' ', x)) 

In [15]:
#save the length of each tweets to df
data['textLength'] = data['text'].str.len()

Verify data integrity


In [21]:
# Select all duplicate rows based on one column
#duplicateRowsDF = data[data.duplicated(['id'])]

#print("Duplicate Rows based on a single column are:", duplicateRowsDF, sep='\n')
##duplicateRowsDF

#Numbers of duplicate rows in each date
#duplicateRowsDFDate = duplicateRowsDF['dateTime'].groupby(duplicateRowsDF['dateTime'].dt.date)
#duplicateRowsDFDate.size()
print(data.shape)

#check if all the text are string
data[data['text'].map(type) == str].shape 



(19936535, 13)


(19936535, 13)

Save processed data


In [22]:
#save the processed data to csv
data.to_csv('../data/processed/bitcoin_tweets_result_processed_20210101-20210930.csv')


Display histograms of tweets length


In [ ]:
#display histograms of tweets length
#cols = ["user_followers", "user_friends", "user_favourites"]

#for col in cols:
col = "textLength"
#btcdata = btcdata.astype({"user_friends": "float64"}) #Conv datatype of df3, user_followers Col from object to int64
#pos_series = btcdata[col].tolist()
#neg_series = negative_examples[col].tolist()
plt.hist(data[col], 50, facecolor='g', alpha=0.7)
#plt.hist(neg_series, 50, facecolor='r', alpha=0.7, label="0")
#plt.legend(prop={"size": 10})
plt.xlabel("Tweet's length", color='#FFC000')
plt.ylabel("Count", color='#FFC000')
plt.ticklabel_format(style='plain')
plt.tick_params(labelcolor='#FFC000')
plt.title("Histogram of tweets length by tweets count", color='#FFC000')
plt.show()

Check the total number of tweets for each day, overlay with bitcoin closing price


In [ ]:
#load bitcoin pricing information
price = pd.read_csv('../data/external/BTC_USD_2020-10-10_2021-10-09-CoinDesk.csv')
pdata = price[['Date', 'Closing Price (USD)']]

In [ ]:
#count number of tweets each date
df = data['dateTime'].groupby(data['dateTime'].dt.date).size().reset_index(name='count')

In [ ]:
#merge two dfs by date


df = df.astype({"dateTime": "str"}) 
print(type(df['dateTime'][0]))
#print(type(pdata['Date'][0]))

#merged_df1 = pd.merge(left=df, right=pdata, left_on='dateTime', right_on='Date')
#merged_df1 = merged_df1.astype({"Closing Price (USD)": "int"}) 
#merged_df1

In [ ]:
#x = merged_df1['dateTime']
#y1 = merged_df1['count']
#y2 = merged_df1['Closing Price (USD)']

x = df['dateTime']
y1 = df['count']

# Plot Line1 (Left Y Axis)
fig, ax1 = plt.subplots(1,1,figsize=(16,9), dpi= 80)
ax1.plot(x, y1, color='tab:red')


# Plot Line2 (Right Y Axis)
#ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
#ax2.plot(x, y2, color='tab:blue')

# Decorations
# ax1 (left Y axis)
ax1.set_xlabel('Date', fontsize=20)
ax1.tick_params(axis='x', rotation=0, labelsize=12)
ax1.set_ylabel('Tweets count by date', color='tab:red', fontsize=20)
ax1.tick_params(axis='y', rotation=0, labelcolor='tab:red' )
ax1.grid(alpha=.4)

ax1.set_xticks(np.arange(0, len(x), 10)) #change this number to change the number of labels
ax1.set_xticklabels(x[::10], rotation=90, fontdict={'fontsize':10}) #change this number to change the number of labels

# ax2 (right Y axis)
#ax2.set_ylabel("Bitcoin closing price (USD)", color='tab:blue', fontsize=20)
#ax2.tick_params(axis='y', labelcolor='tab:blue')
#ax2.set_xticks(np.arange(0, len(x), 20)) #change this number to change the number of labels
#ax2.set_xticklabels(x[::20], rotation=90, fontdict={'fontsize':10}) #change this number to change the number of labels
#ax2.set_title("Personal Savings Rate vs Unemployed: Plotting in Secondary Y Axis", fontsize=22)
fig.tight_layout()
plt.show()




Randomly select rows from each day


In [ ]:
#btcdata.groupby(btcdata['date_time'].dt.year).apply(lambda x :x.iloc[random.choice(range(0,len(x)))])
#btcdata.groupby(btcdata['date_time'].dt.year).apply(lambda x: x.sample(2)).reset_index(drop=True)

selected = data.groupby(data['dateTime'].dt.day).apply(lambda x: x.sample(10, random_state=889))

selected.to_csv('sample_bitcoin_tweets_ten_each_day.csv')
selected